Import packages

In [108]:
import librosa.feature
import pandas as pd
import numpy as np
import seaborn as sns
import dython as dy
from os.path import join
import scipy.io.wavfile as wav
import librosa as lbr
from scipy import signal
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from sklearn.model_selection import KFold, ParameterGrid
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
import noisereduce as nr
import csv
import warnings
warnings.filterwarnings('ignore')

Spectrometer function

In [143]:
def spect(arr, fs, n_sub_r,n_sub_c):
    spect_list = []
    for i in range(0,len(arr)):
        # freq, time,  spectrum = signal.spectrogram(x=arr[i], fs=fs[i])
        spectrum = lbr.feature.melspectrogram(arr[i],sr=fs[i])
        # spectrum = 10*np.log10(spectrum)
        spect_split1 = np.array_split(spectrum, n_sub_r, axis=0)
        tmp_row = []
        for row in spect_split1:
            spect_split2 = np.array_split(row, n_sub_c, axis=1)
            for block in spect_split2:
                tmp_row.append(block.mean())
        spect_list.append(tmp_row)
        if i%100 == 0:
            print(i)
    print(np.array(spect_list).shape)
    return spect_list

In [142]:
def data_import(path,par='dev'):
    df_in = pd.read_csv(path)
    df_in = df_in.rename(columns={'Self-reported fluency level ': 'fluency', 'First Language spoken': 'firstLan', 'Current language used for work/school': 'currentLan'})
    df_out = df_in[['fluency','gender','currentLan']]
    X = []
    y = []
    rate_list = []

    for item in df_in.iterrows():
        filename = item[1]['path']
        data, rate = lbr.load(filename, sr= None)
        data_no_silence = lbr.effects.trim(data, top_db= 10)
        data_emp = lbr.effects.preemphasis(data_no_silence[0])
        data_processed = nr.reduce_noise(y=data_emp, sr=rate)
        rate_list.append(rate)
        X.append(data_processed)
        if par=='dev':
            label = item[1]['action']+item[1]['object']
            y.append(label)

    if par=='dev':
        return df_out,X,y,rate_list
    else:
        return df_out,X,rate_list

Import data

In [144]:
df_dev,X_dev_list,y_dev_list,rate_dev= data_import('development.csv','dev')
df_ev,X_ev_list,rate_ev = data_import('evaluation.csv','ev')

print(len(X_dev_list))
print(len(X_ev_list))

9854
1455


Pre-processing

In [150]:
n_sub_row = 10
n_sub_col = 10
np.seterr(divide = 'ignore')

spectX_dev_list = spect(X_dev_list,rate_dev, n_sub_row, n_sub_col)
print('Spectrometer conversion X_dev: DONE')
spectX_ev_list = spect(X_ev_list, rate_ev, n_sub_row, n_sub_col)
print('Spectrometer conversion X_ev: DONE')

y_dev = pd.DataFrame(np.array(y_dev_list).transpose())
X_dev = pd.DataFrame(spectX_dev_list).fillna(0)
X_ev = pd.DataFrame(spectX_ev_list).fillna(0)
print('Dataframe conversion: DONE')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
(9854, 100)
Spectrometer conversion X_dev: DONE
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
(1455, 100)
Spectrometer conversion X_ev: DONE
Dataframe conversion: DONE


In [64]:
X_dev_n = normalize(X_dev)
X_ev_n = normalize(X_ev)
print('Normalization: DONE')


Normalization: DONE


In [ ]:
tot_attr = n_sub_row*n_sub_col
max_attr = 300
pca = PCA(n_components=tot_attr)
X_proj = pca.fit_transform(X_dev)
plt.plot(pca.explained_variance_ratio_, marker='o', linestyle='')

pca = PCA(n_components=max_attr)
X_dev_proj = pca.fit_transform(X_dev)
X_ev_proj = pca.transform(X_ev)
X_dev_proj=pd.DataFrame(X_dev_proj)
print(X_dev_proj.info())

In [151]:
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X_dev, y_dev, test_size=0.3)

Grid-search

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {
    "n_estimators": [50, 100, 200, 500],
    "criterion": ["gini", "entropy"]
    }


grid_search = GridSearchCV(RandomForestClassifier(random_state=42), params,cv=5, scoring="accuracy")
grid_search.fit(X_train_valid,y_train_valid)
print(grid_search.best_params_, '->', grid_search.best_score_)

In [ ]:
params = {
    "n_estimators": [50, 100, 200, 500],
    "criterion": ["gini", "entropy"]
    }


grid_search = GridSearchCV(ExtraTreesClassifier(random_state=42), params,cv=5, scoring="accuracy")
grid_search.fit(X_train_valid,y_train_valid)
print(grid_search.best_params_, '->', grid_search.best_score_)

In [ ]:
from sklearn.svm import SVC

params = {
    "kernel": ["linear", "poly", "rbf"]
    }


grid_search = GridSearchCV(SVC(random_state=42), params,cv=5, scoring="accuracy")
grid_search.fit(X_train_valid,y_train_valid)
print(grid_search.best_params_, '->', grid_search.best_score_)

Test

In [125]:
rf = RandomForestClassifier(n_estimators=1000, criterion='entropy')
rf.fit(X_train_valid, y_train_valid.values.ravel())
y_prev = rf.predict(X_test)
score1 = f1_score(y_true=y_test, y_pred=y_prev, average='macro')
score2 = accuracy_score(y_true=y_test, y_pred=y_prev)
print(score1," - ",score2)

0.6871337244116653  -  0.6940639269406392


In [147]:
rf = ExtraTreesClassifier(n_estimators=500,criterion='entropy')
rf.fit(X_train_valid, y_train_valid.values.ravel())
y_prev = rf.predict(X_test)
score1 = f1_score(y_true=y_test, y_pred=y_prev, average='macro')
score2 = accuracy_score(y_true=y_test, y_pred=y_prev)
print(score1," - ",score2)

0.639064956966978  -  0.6504312531709792


Evaluation

In [148]:
rf = RandomForestClassifier(n_estimators=1000,criterion='entropy')
rf.fit(X_dev, y_dev.values.ravel())
y_prev_ev = rf.predict(X_ev)

Generate results

In [149]:
with open('res.csv', mode='w', newline='') as file:
    file_writer = csv.writer(file)
    i = 0
    file_writer.writerow(['Id', 'Predicted'])
    for elem in y_prev_ev:
        file_writer.writerow([i, elem])
        i += 1